# HackMIT19 Puzzle 6

## Imports

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import cifar
import numpy as np
import os
import reqd

Using TensorFlow backend.


## Constants

In [2]:
num_classes = 100
epochs = 15
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar100_trained_model.h5'
model_path = os.path.join(save_dir, model_name)

## Load cifar100 images
Load cifar 100 images and divide into 50/50 train test randomly

In [3]:
# Load all cifar images
cimgs, clbls = cifar.load()
assert(len(set(clbls))==num_classes) # Ensure all labels are present
Ncimgs = len(cimgs)

# Convert image matrices to float32 and normalise
cimgs = np.array(cimgs, dtype='float32')
cimgs /= 255
print clbls.dtype

int64


## Function to return random 50/50 split of CIFAR-100 images

In [4]:
def randimgs():
    # random divide into train/test
    perm = np.random.permutation(np.arange(Ncimgs))

    train_inds = perm[:Ncimgs/2]
    test_inds = perm[Ncimgs/2:]
    
    train_x = cimgs[train_inds]
    train_y = clbls[train_inds]
    test_x = cimgs[test_inds]
    test_y = clbls[test_inds]
    assert(len(train_x)==len(test_x) and len(train_x) + len(test_y) == Ncimgs)

    # Convert class vectors to binary class matrices.
    train_y = keras.utils.to_categorical(train_y, num_classes)
    test_y = keras.utils.to_categorical(test_y, num_classes)
    print test_y.dtype
    
    assert(train_y.shape == (Ncimgs/2, num_classes))
    return (train_x, train_y), (test_x, test_y), (train_inds, test_inds)

## Construct HackMIT model
Construct the HackMIT model as given in the description of the puzzle

In [5]:
def createMITModel():
    # Construct model
    mitmdl = Sequential()
    mitmdl.add(Conv2D(64, (3, 3), input_shape=(32,32,3), padding='same', activation='relu'))
    mitmdl.add(Conv2D(64, (3, 3), activation='relu'))
    mitmdl.add(MaxPooling2D(pool_size=(2, 2)))
    mitmdl.add(Flatten())
    mitmdl.add(Dense(512, activation='relu'))
    mitmdl.add(Dense(256, activation='relu'))
    mitmdl.add(Dense(100, activation='softmax'))
    # compile model
    mitmdl.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return mitmdl

## Function to generate input samples for classification
Creates a model to the HackMIT specification and runs a random split of cifar images through it. Then creates the input and expected output for binary classifcation of test/training

In [6]:
def getSamples(batch_size=128):
    mitmdl = createMITModel()
    (train_x, train_y), (test_x, test_y), (train_inds, test_inds) = randimgs()
    mitmdl.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_x, test_y),
          shuffle=True)
    
    outs = mitmdl.predict(cimgs)
    ln = len(outs)
    X = np.empty((ln, 200))
    y = np.empty(ln)
    cnt = 0
    for x in train_inds:
        tgt = np.zeros(100, dtype='float32')
        tgt[clbls[x]] = 1.0
        X[cnt, :] = np.concatenate((outs[x, :], tgt))
        y[cnt] = 1
        cnt += 1

    for x in test_inds:
        tgt = np.zeros(100, dtype='float32')
        tgt[clbls[x]] = 1.0
        X[cnt, :] = np.concatenate((outs[x, :], tgt))
        y[cnt] = 0
        cnt += 1

    assert(cnt == ln)

    y = keras.utils.to_categorical(y, 2)
    return X, y

In [7]:
nsamples = 2
xs = []
ys = []
for i in xrange(nsamples):
    X, y = getSamples()
    xs.append(X)
    ys.append(y)

X = np.concatenate(xs)
y = np.concatenate(ys)
print X.shape, y.shape
ln = len(X)
print "Length", ln

# Random shuffle
TSTcnt = int(ln*0.15)
print "Tst count", TSTcnt
idx = np.random.permutation(np.arange(ln))
Xtrain = X[idx[TSTcnt:]]
ytrain = y[idx[TSTcnt:]]
Xtst = X[idx[:TSTcnt]]
ytst = y[idx[:TSTcnt]]

Instructions for updating:
Colocations handled automatically by placer.
float32
Instructions for updating:
Use tf.cast instead.
Train on 30000 samples, validate on 30000 samples
Epoch 1/15
30000/30000 [==============================] - 25s 820us/step - loss: 3.9211 - acc: 0.1063 - val_loss: 3.4337 - val_acc: 0.1802
Epoch 2/15
30000/30000 [==============================] - 21s 683us/step - loss: 3.0856 - acc: 0.2454 - val_loss: 3.0165 - val_acc: 0.2635
Epoch 3/15
30000/30000 [==============================] - 20s 672us/step - loss: 2.5484 - acc: 0.3555 - val_loss: 2.8250 - val_acc: 0.3043
Epoch 4/15
30000/30000 [==============================] - 23s 765us/step - loss: 2.0335 - acc: 0.4655 - val_loss: 2.8563 - val_acc: 0.3126
Epoch 5/15
30000/30000 [==============================] - 20s 657us/step - loss: 1.5015 - acc: 0.5904 - val_loss: 2.9889 - val_acc: 0.3177
Epoch 6/15
30000/30000 [==============================] - 21s 694us/step - loss: 0.9737 - acc: 0.7293 - val_loss: 3.4217 - val_

In [8]:
model = Sequential()
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(2, activation='softmax'))
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(Xtrain, ytrain, batch_size=100, epochs=15, shuffle=True, validation_data=(Xtst,ytst))

Train on 102000 samples, validate on 18000 samples
Epoch 1/15
102000/102000 [==============================] - 4s 35us/step - loss: 0.3773 - acc: 0.8385 - val_loss: 0.3512 - val_acc: 0.8538
Epoch 2/15
102000/102000 [==============================] - 3s 29us/step - loss: 0.3376 - acc: 0.8591 - val_loss: 0.3473 - val_acc: 0.8529
Epoch 3/15
102000/102000 [==============================] - 3s 31us/step - loss: 0.3329 - acc: 0.8601 - val_loss: 0.3482 - val_acc: 0.8519
Epoch 4/15
102000/102000 [==============================] - 3s 30us/step - loss: 0.3288 - acc: 0.8618 - val_loss: 0.3479 - val_acc: 0.8541
Epoch 5/15
102000/102000 [==============================] - 3s 30us/step - loss: 0.3270 - acc: 0.8624 - val_loss: 0.3473 - val_acc: 0.8535
Epoch 6/15
102000/102000 [==============================] - 3s 30us/step - loss: 0.3245 - acc: 0.8634 - val_loss: 0.3451 - val_acc: 0.8558
Epoch 7/15
102000/102000 [==============================] - 3s 31us/step - loss: 0.3216 - acc: 0.8642 - val_loss: 0